## 第2章: UNIXコマンドの基礎
popular-names.txtは，アメリカで生まれた赤ちゃんの「名前」「性別」「人数」「年」をタブ区切り形式で格納したファイルである．以下の処理を行うプログラムを作成し，popular-names.txtを入力ファイルとして実行せよ．さらに，同様の処理をUNIXコマンドでも実行し，プログラムの実行結果を確認せよ．

In [2]:
!wget -nc https://nlp100.github.io/data/popular-names.txt -P data/

--2021-05-30 22:37:26--  https://nlp100.github.io/data/popular-names.txt
nlp100.github.io (nlp100.github.io) をDNSに問いあわせています... 185.199.110.153, 185.199.109.153, 185.199.111.153, ...
nlp100.github.io (nlp100.github.io)|185.199.110.153|:443 に接続しています... 接続しました。
HTTP による接続要求を送信しました、応答を待っています... 200 OK
長さ: 55026 (54K) [text/plain]
`data/popular-names.txt' に保存中

popular-names.txt   100%[===================>]  53.74K  --.-KB/s 時間 0.01s      

2021-05-30 22:37:26 (3.68 MB/s) - `data/popular-names.txt' へ保存完了 [55026/55026]



## 10. 行数のカウント
行数をカウントせよ．確認にはwcコマンドを用いよ．

In [4]:
path = 'data/popular-names.txt'
print(sum([1 for _ in open(path)]))

2780


In [6]:
!wc data/popular-names.txt

 2780 11120 55026 data/popular-names.txt


In [8]:
# 行数のみを出力
!cat data/popular-names.txt | wc -l

2780


- grep '正規表現'
  - 正規表現で与えられた文字列を検索

In [6]:
!grep Alice data/popular-names.txt

Alice	F	1414	1880
Alice	F	1308	1881
Alice	F	1542	1882
Alice	F	3135	1902
Alice	F	3610	1905
Alice	F	4192	1906
Alice	F	4107	1907
Alice	F	4270	1908
Alice	F	4170	1909
Alice	F	4671	1910


## 11.タブをスペースに変換
タブ1文字につきスペース1文字に置換せよ．確認にはsedコマンド，trコマンド，もしくはexpandコマンドを用いよ．

In [11]:
from itertools import islice

with open(path) as fi:
    for line in islice(fi, 10):
        line = line.strip()
        print(line.replace('\t', ' '))

Mary F 7065 1880
Anna F 2604 1880
Emma F 2003 1880
Elizabeth F 1939 1880
Minnie F 1746 1880
Margaret F 1578 1880
Ida F 1472 1880
Alice F 1414 1880
Bertha F 1320 1880
Sarah F 1288 1880


In [12]:
!tr '$\t' " " < data/popular-names.txt | head

Mary F 7065 1880
Anna F 2604 1880
Emma F 2003 1880
Elizabeth F 1939 1880
Minnie F 1746 1880
Margaret F 1578 1880
Ida F 1472 1880
Alice F 1414 1880
Bertha F 1320 1880
Sarah F 1288 1880
tr: write error: Broken pipe


In [14]:
!expand -t 1 < data/popular-names.txt | head

Mary F 7065 1880
Anna F 2604 1880
Emma F 2003 1880
Elizabeth F 1939 1880
Minnie F 1746 1880
Margaret F 1578 1880
Ida F 1472 1880
Alice F 1414 1880
Bertha F 1320 1880
Sarah F 1288 1880
expand: write error: Broken pipe
expand: write error


In [15]:
!sed s/'\t'/' '/g data/popular-names.txt 2>/dev/null | head

Mary F 7065 1880
Anna F 2604 1880
Emma F 2003 1880
Elizabeth F 1939 1880
Minnie F 1746 1880
Margaret F 1578 1880
Ida F 1472 1880
Alice F 1414 1880
Bertha F 1320 1880
Sarah F 1288 1880


In [16]:
# sedコマンドは[s]の直後の文字を自動的に区切り文字とみなすので[!]にしてもOK
!sed s!'\t'!' '!g data/popular-names.txt 2>/dev/null | head

Mary F 7065 1880
Anna F 2604 1880
Emma F 2003 1880
Elizabeth F 1939 1880
Minnie F 1746 1880
Margaret F 1578 1880
Ida F 1472 1880
Alice F 1414 1880
Bertha F 1320 1880
Sarah F 1288 1880


## 12. 1列目をcol1.txtに，2列目をcol2.txtに保存
各行の1列目だけを抜き出したものをcol1.txtに，2列目だけを抜き出したものをcol2.txtとしてファイルに保存せよ．確認にはcutコマンドを用いよ．

In [17]:
with open(path) as fi, \
    open('work/col1.txt', 'w') as col1, \
    open('work/col2.txt', 'w') as col2:
    for line in fi:
        name, gender, *_ = line.strip().split('\t')
        print(name, file=col1)
        print(gender, file=col2)

In [20]:
!head work/col1.txt

Mary
Anna
Emma
Elizabeth
Minnie
Margaret
Ida
Alice
Bertha
Sarah


In [18]:
!cut -f 1 data/popular-names.txt | head

Mary
Anna
Emma
Elizabeth
Minnie
Margaret
Ida
Alice
Bertha
Sarah


In [21]:
!head work/col2.txt

F
F
F
F
F
F
F
F
F
F


In [19]:
!cut -f 2 data/popular-names.txt | head

F
F
F
F
F
F
F
F
F
F


## 13. col1.txtとcol2.txtをマージ
12で作ったcol1.txtとcol2.txtを結合し，元のファイルの1列目と2列目をタブ区切りで並べたテキストファイルを作成せよ．確認にはpasteコマンドを用いよ．

In [22]:
with open('work/col1.txt') as col1, \
    open('work/col2.txt') as col2, \
    open('work/col13.txt', 'w') as col13:
    for name, gender in zip(col1,col2):  # lineのみだとlineにcol1,col2の要素がリストになってループする
        print(name.strip() + '\t' + gender.strip(), file=col13)

In [23]:
!head work/col13.txt

Mary	F
Anna	F
Emma	F
Elizabeth	F
Minnie	F
Margaret	F
Ida	F
Alice	F
Bertha	F
Sarah	F


In [24]:
!paste work/col1.txt work/col2.txt | head

Mary	F
Anna	F
Emma	F
Elizabeth	F
Minnie	F
Margaret	F
Ida	F
Alice	F
Bertha	F
Sarah	F


## 14. 先頭からN行を出力
自然数Nをコマンドライン引数などの手段で受け取り，入力のうち先頭のN行だけを表示せよ．確認にはheadコマンドを用いよ．

In [27]:
%%file src/q14.py
import sys


def begin_n_line(n, path):
    with open(path) as fi:
        print(''.join([fi.readline() for i in range(n)]))
        
    
if __name__ == "__main__":
    if len(sys.argv) != 3:
        print('コマンドライン引数が不適切です')
        print('python q14.py [出力したい行数] [入力ファイル]')
    begin_n_line(int(sys.argv[1]), sys.argv[2])

Overwriting src/q14.py


In [28]:
!python src/q14.py 2 data/popular-names.txt

Mary	F	7065	1880
Anna	F	2604	1880



In [29]:
!head -n 2 data/popular-names.txt

Mary	F	7065	1880
Anna	F	2604	1880


## 15. 末尾のN行を出力
自然数Nをコマンドライン引数などの手段で受け取り，入力のうち末尾のN行だけを表示せよ．確認にはtailコマンドを用いよ．

In [44]:
%%file src/q15.py
import sys
from collections import deque

def end_n_line(n, path):
    with open(path) as fi:
        for line in deque(fi, n):
            print(line, end='')
    

if __name__ == "__main__":
    if len(sys.argv) != 3:
        print('コマンドライン引数が不適切です')
        print('python q14.py [出力したい行数] [入力ファイル]')
    end_n_line(int(sys.argv[1]), sys.argv[2])

Overwriting src/q15.py


In [45]:
!python src/q15.py 3 data/popular-names.txt

Lucas	M	12585	2018
Mason	M	12435	2018
Logan	M	12352	2018


In [46]:
!tail -n 3 data/popular-names.txt

Lucas	M	12585	2018
Mason	M	12435	2018
Logan	M	12352	2018


## 16. ファイルをN分割する
自然数Nをコマンドライン引数などの手段で受け取り，入力のファイルを行単位でN分割せよ．同様の処理をsplitコマンドで実現せよ．

In [47]:
%%file src/q16.py
import sys


def split_num(n, path):
    # 分割ファイル１つあたりの行数を返す関数
    split_list = []
    file_lines = sum([1 for _ in open(path)])  # ファイルの行数のカウント
    q, mod = divmod(file_lines, n)
    for i in range(n):  # 分割されたものにそれぞれ何行入るかのリストを作成 n=4ならsplit_list=[6,6,6,6]
        split_list.append(q)
    for i in range(mod):  # 余りを一つずつ配分していく
        split_list[i] += 1
    return split_list


def split(split_list, path):
    # 分割ファイル１つあたりの行数のリストを引数に分割したものを出力
    with open(path) as fi:
        for split in split_list:
            for i in range(split):
                print(fi.readline().strip())
            print("-----")

            
if __name__ == "__main__":
    if len(sys.argv) != 3:
        print('コマンドライン引数が不適切です')
        print('python q14.py [出力したい行数] [入力ファイル]')
    lis = split_num(int(sys.argv[1]), sys.argv[2])
    split(lis, sys.argv[2])

Writing src/q16.py


In [50]:
!python src/q16.py 900 data/popular-names.txt 2>/dev/null | head

Mary	F	7065	1880
Anna	F	2604	1880
Emma	F	2003	1880
Elizabeth	F	1939	1880
-----
Minnie	F	1746	1880
Margaret	F	1578	1880
Ida	F	1472	1880
Alice	F	1414	1880
-----


In [1]:
# [-l　行数]で分割ファイルの行数を指定、[-n　個数]でファイルの個数を指定して分割
!split -n 3 data/popular-names.txt work/q16.txt

## 17. １列目の文字列の異なり
1列目の文字列の種類（異なる文字列の集合）を求めよ．確認にはcut, sort, uniqコマンドを用いよ．

In [34]:
with open('data/popular-names.txt') as fi:
    names = set()
    for line in fi:
        names.add(line.strip().split()[0])

print(sorted(names))

['Abigail', 'Aiden', 'Alexander', 'Alexis', 'Alice', 'Amanda', 'Amelia', 'Amy', 'Andrew', 'Angela', 'Anna', 'Annie', 'Anthony', 'Ashley', 'Austin', 'Ava', 'Barbara', 'Benjamin', 'Bertha', 'Bessie', 'Betty', 'Brandon', 'Brian', 'Brittany', 'Carol', 'Carolyn', 'Charles', 'Charlotte', 'Chloe', 'Christopher', 'Clara', 'Crystal', 'Cynthia', 'Daniel', 'David', 'Deborah', 'Debra', 'Donald', 'Donna', 'Doris', 'Dorothy', 'Edward', 'Elijah', 'Elizabeth', 'Emily', 'Emma', 'Ethan', 'Ethel', 'Evelyn', 'Florence', 'Frances', 'Frank', 'Gary', 'George', 'Hannah', 'Harper', 'Harry', 'Heather', 'Helen', 'Henry', 'Ida', 'Isabella', 'Jacob', 'James', 'Jason', 'Jayden', 'Jeffrey', 'Jennifer', 'Jessica', 'Joan', 'John', 'Joseph', 'Joshua', 'Judith', 'Julie', 'Justin', 'Karen', 'Kathleen', 'Kelly', 'Kimberly', 'Larry', 'Laura', 'Lauren', 'Liam', 'Lillian', 'Linda', 'Lisa', 'Logan', 'Lori', 'Lucas', 'Madison', 'Margaret', 'Marie', 'Mark', 'Mary', 'Mason', 'Matthew', 'Megan', 'Melissa', 'Mia', 'Michael', 'Mich

In [32]:
!cut -f 1 data/popular-names.txt | sort -u | head -5

Abigail
Aiden
Alexander
Alexis
Alice


In [33]:
!cut -f 1 data/popular-names.txt | sort | uniq | head -5

Abigail
Aiden
Alexander
Alexis
Alice


## 18. 各行を3コラム目の数値の降順にソート
各行を3コラム目の数値の逆順で整列せよ（注意: 各行の内容は変更せずに並び替えよ）．確認にはsortコマンドを用いよ（この問題はコマンドで実行した時の結果と合わなくてもよい）．

In [37]:
data = [line.strip().split() for line in open(path)]
data = sorted(data, key=lambda x:int(x[2]), reverse=True)
data[:10]

[['Linda', 'F', '99689', '1947'],
 ['Linda', 'F', '96211', '1948'],
 ['James', 'M', '94757', '1947'],
 ['Michael', 'M', '92704', '1957'],
 ['Robert', 'M', '91640', '1947'],
 ['Linda', 'F', '91016', '1949'],
 ['Michael', 'M', '90656', '1956'],
 ['Michael', 'M', '90517', '1958'],
 ['James', 'M', '88584', '1948'],
 ['Michael', 'M', '88528', '1954']]

sort コマンドのいろいろなオプション
- -k 列番号: 指定した列番号の列をキーに使う
- -r: 逆順（降順）にソート
- -n: 数値としてソート．デフォルトでは辞書順にソートされる

In [38]:
!sort -k 3 -rn data/popular-names.txt 2>/dev/null | head

Linda	F	99689	1947
Linda	F	96211	1948
James	M	94757	1947
Michael	M	92704	1957
Robert	M	91640	1947
Linda	F	91016	1949
Michael	M	90656	1956
Michael	M	90517	1958
James	M	88584	1948
Michael	M	88528	1954


## 19. 各行の1コラム目の文字列の出現頻度を求め，出現頻度の高い順に並べる
各行の1列目の文字列の出現頻度を求め，その高い順に並べて表示せよ．確認にはcut, uniq, sortコマンドを用いよ．

In [39]:
from collections import Counter
name_counter = Counter(line.strip().split()[0] for line in open(path))
name_counter.most_common(10)

[('James', 118),
 ('William', 111),
 ('John', 108),
 ('Robert', 108),
 ('Mary', 92),
 ('Charles', 75),
 ('Michael', 74),
 ('Elizabeth', 73),
 ('Joseph', 70),
 ('Margaret', 60)]

In [41]:
from collections import defaultdict
names = defaultdict(int)
for line in open(path):
    names[line.strip().split()[0]] += 1
sorted_names = sorted(names.items(), key=lambda item:item[1], reverse=True)
sorted_names[:10]

[('James', 118),
 ('William', 111),
 ('John', 108),
 ('Robert', 108),
 ('Mary', 92),
 ('Charles', 75),
 ('Michael', 74),
 ('Elizabeth', 73),
 ('Joseph', 70),
 ('Margaret', 60)]

In [40]:
!cut -f 1 data/popular-names.txt | sort | uniq -c |sort -nr | head

    118 James
    111 William
    108 Robert
    108 John
     92 Mary
     75 Charles
     74 Michael
     73 Elizabeth
     70 Joseph
     60 Margaret
